## Setting up

In [1]:
%%capture
%pip install -U bitsandbytes
%pip install -U transformers
%pip install -U accelerate
%pip install -U peft
%pip install -U trl

In [ ]:
import wandb
wandb.init(mode="disabled")

from huggingface_hub import login
api_token = 'give_our_own_toke'
login(api_token)

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: fineGrained).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [3]:
import numpy as np
import pandas as pd
import os
from tqdm import tqdm
import bitsandbytes as bnb
import torch
import torch.nn as nn
import transformers
from datasets import Dataset
from peft import LoraConfig, PeftConfig, get_peft_model, PeftModel
from trl import SFTTrainer
from trl import setup_chat_format
from transformers import (AutoModelForCausalLM, 
                          AutoTokenizer, 
                          BitsAndBytesConfig, 
                          TrainingArguments, 
                          pipeline, 
                          logging)
from sklearn.metrics import (accuracy_score, 
                             classification_report, 
                             confusion_matrix)
from sklearn.model_selection import train_test_split

## Loading and processing the dataset

In [ ]:
# df = pd.read_csv("/kaggle/input/sentiment-analysis-for-mental-health/Combined Data.csv",index_col = "Unnamed: 0") s
# df.loc[:,'status'] = df.loc[:,'status'].str.replace('Bi-Polar','Bipolar')
# df = df[(df.status != "Personality disorder") & (df.status != "Stress") & (df.status != "Suicidal")]
# df.head()

In [5]:
sampled_df = pd.read_csv('/kaggle/input/dlsprint3/train.csv')
percentage = 1 

# Sample x percent of the DataFrame
df = sampled_df.sample(frac=percentage, random_state=42)  # random_state for reproducibility

df

,ID,Problem,Answer
30,30,"0, 1, 2, 7, 8, 9 অংকগুলো মাত্র একবার ব্যবহার ক...",100
171,171,সামিরা বর্ণমালা এবং সংখ্যা ব্যবহার করে মজার কি...,1191911
84,84,এমন সম্ভাব্য সকল ধনাত্মক পূর্ণসংখ্যা $x$ এর যো...,46
198,198,"তুমি কিছু সংখ্যক চকলেট ও আইসক্রিম কিনলে, যেখান...",216
60,60,"একদিন তোমার মনে হল, তুমি খেজুরের রস খেতে চাও। ...",36
...,...,...,...
106,106,সাদিয়ার কী-বোর্ডে 0 থেকে 9 পর্যন্ত সংখ্যার বোত...,3439
14,14,"নিলয়, তাহমিদ ও জ্যোতি প্রত্যেকে যথাক্রমে লাল, ...",27
92,92,"\(f(n) = 3f(n + 1) - 2f(n - 1)\), \(f(0)=0\) এ...",3
179,179,13 এর সকল উৎপাদকগুলোর যোগফল কত?,14


In [6]:
# Shuffle the DataFrame and select only 3000 rows
df = df.sample(frac=1, random_state=85).reset_index(drop=True)

# Split the DataFrame
train_size = 0.8
eval_size = 0.1

# Calculate sizes
train_end = int(train_size * len(df))
eval_end = train_end + int(eval_size * len(df))

# Split the data
X_train = df[:train_end]
X_eval = df[train_end:eval_end]
X_test = df[eval_end:]


# input_text = f"You are a model smarter than Einstein when it comes to solving mathematical problems. \
#          You are also skilled at solving Bengali problems. Here is a Bengali problem: \
#          Problem: {instruction}\nPlease solve the problem step by step and enclose the final result in \\boxed{{}}."


System_message= "You are a model smarter than Einstein when it comes to solving mathematical problems. You are also skilled at solving Bengali problems."

# Define the prompt generation functions
def generate_prompt(data_point):
    return f"""
    {System_message}
    Here is a Bengali problem:

    Problem: {data_point["Problem"]} \nPlease solve the problem step by step and enclose the final result in \\boxed{{}}.
    label: {data_point["Answer"]}""".strip()

def generate_test_prompt(data_point):
    return f"""
    {System_message}
    Here is a Bengali problem:

    Problem: {data_point["Problem"]} \nPlease solve the problem step by step and enclose the final result in \\boxed{{}}.
    label: """.strip()

# Generate prompts for training and evaluation data
X_train.loc[:,'text'] = X_train.apply(generate_prompt, axis=1)
X_eval.loc[:,'text'] = X_eval.apply(generate_prompt, axis=1)

X_test = X_test.sample(frac=0.3, random_state=42)  # random_state for reproducibility

# Generate test prompts and extract true labels
y_true = X_test.loc[:,'Answer']
X_test = pd.DataFrame(X_test.apply(generate_test_prompt, axis=1), columns=["text"])



/tmp/ipykernel_30/315220117.py:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_train.loc[:,'text'] = X_train.apply(generate_prompt, axis=1)
/tmp/ipykernel_30/315220117.py:44: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_eval.loc[:,'text'] = X_eval.apply(generate_prompt, axis=1)


In [7]:
X_train.iloc[0].text

'You are a model smarter than Einstein when it comes to solving mathematical problems. You are also skilled at solving Bengali problems.\n    Here is a Bengali problem:\n\n    Problem: \\( (2 + 1)(2^2 + 1)(2 ^{2^2} + 1) \\dots (2^{2^{10}} + 1) + 1 = 2^n \\) হলে, \\( n \\) এর মান নির্ণয় করো। \nPlease solve the problem step by step and enclose the final result in \\boxed{}.\n    label: 2048'

In [8]:
y_true

187       2
200    2152
195       5
188       6
202      13
192      21
207    1025
Name: Answer, dtype: int64

In [9]:
# Convert to datasets
train_data = Dataset.from_pandas(X_train[["text"]])
eval_data = Dataset.from_pandas(X_eval[["text"]])

In [10]:
train_data['text'][3]

'You are a model smarter than Einstein when it comes to solving mathematical problems. You are also skilled at solving Bengali problems.\n    Here is a Bengali problem:\n\n    Problem: শাকুর একটি অনুক্রম তৈরি করলো যার $n$-তম পদ হচ্ছে প্রথম $n$ টি স্বাভাবিক সংখ্যার সমষ্টি। শাকুরের অনুক্রমের প্রথম 99 টি পদের সমষ্টি কত? \nPlease solve the problem step by step and enclose the final result in \\boxed{}.\n    label: 166650'

## Loading the model and tokenizer

In [11]:
base_model_name = "AI-MO/NuminaMath-7B-TIR"


bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=False,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype="float16",
)

# bnb_config = BitsAndBytesConfig(
#     load_in_8bit=True,  # Enable 8-bit quantization
#     bnb_8bit_compute_dtype=torch.bfloat16  # Use bfloat16 for computation (can also use float16 or float32)
# )

model = AutoModelForCausalLM.from_pretrained(
    base_model_name,
    device_map="auto",
    torch_dtype="float16",
    quantization_config=bnb_config, 
)

model.config.use_cache = False
model.config.pretraining_tp = 1

config.json:   0%|          | 0.00/716 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/22.5k [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.99G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/3.85G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/121 [00:00<?, ?B/s]

In [12]:
tokenizer = AutoTokenizer.from_pretrained(base_model_name)

tokenizer.pad_token_id = tokenizer.eos_token_id

tokenizer_config.json:   0%|          | 0.00/1.23k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/4.61M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/482 [00:00<?, ?B/s]

## Model evalution before fine-tuning

In [13]:
def predict(test, model, tokenizer):
    y_pred = []
    categories = ["human", "machine"]
    
    for i in tqdm(range(len(test))):
        prompt = test.iloc[i]["text"]
        pipe = pipeline(task="text-generation", 
                        model=model, 
                        tokenizer=tokenizer, 
                        max_new_tokens=20, 
                        temperature=0.4)
        
        result = pipe(prompt)
#         print(result)
        answer = result[0]['generated_text'].split("label:")[-1].strip()
        
        # Determine the predicted category
        for category in categories:
            if category.lower() in answer.lower():
                y_pred.append(category)
                break
        else:
            y_pred.append("none")
    
    return y_pred

In [14]:
def evaluate(y_true, y_pred):
    labels = ["human", "machine"]
    mapping = {label: idx for idx, label in enumerate(labels)}
    
    def map_func(x):
        return mapping.get(x, -1)  # Map to -1 if not found, but should not occur with correct data
    
    y_true_mapped = np.vectorize(map_func)(y_true)
    y_pred_mapped = np.vectorize(map_func)(y_pred)
    
    # Calculate accuracy
    accuracy = accuracy_score(y_true=y_true_mapped, y_pred=y_pred_mapped)
    print(f'Accuracy: {accuracy:.3f}')
    
    # Generate accuracy report
    unique_labels = set(y_true_mapped)  # Get unique labels
    
    for label in unique_labels:
        label_indices = [i for i in range(len(y_true_mapped)) if y_true_mapped[i] == label]
        label_y_true = [y_true_mapped[i] for i in label_indices]
        label_y_pred = [y_pred_mapped[i] for i in label_indices]
        label_accuracy = accuracy_score(label_y_true, label_y_pred)
        print(f'Accuracy for label {labels[label]}: {label_accuracy:.3f}')
    
    # Generate classification report
    class_report = classification_report(y_true=y_true_mapped, y_pred=y_pred_mapped, target_names=labels, labels=list(range(len(labels))))
    print('\nClassification Report:')
    print(class_report)
    
    # Generate confusion matrix
    conf_matrix = confusion_matrix(y_true=y_true_mapped, y_pred=y_pred_mapped, labels=list(range(len(labels))))
    print('\nConfusion Matrix:')
    print(conf_matrix)

In [15]:

# y_pred = predict(X_test, model, tokenizer)
# print(y_pred)
# evaluate(y_true, y_pred)

## Extracting the linear modules names

In [16]:
import bitsandbytes as bnb

def find_all_linear_names(model):
    cls = bnb.nn.Linear4bit
    lora_module_names = set()
    for name, module in model.named_modules():
        if isinstance(module, cls):
            names = name.split('.') 
            lora_module_names.add(names[0] if len(names) == 1 else names[-1])
    if 'lm_head' in lora_module_names:  
        lora_module_names.remove('lm_head')
    return list(lora_module_names)

In [17]:
modules = find_all_linear_names(model)
print(modules)

modules = ['v_proj', 'down_proj', 'k_proj', 'up_proj', 'q_proj', 'o_proj', 'gate_proj']

['v_proj', 'up_proj', 'q_proj', 'down_proj', 'k_proj', 'o_proj', 'gate_proj']


## Setting up the model

In [18]:
output_dir="NuminaMath-7B-TIR"

peft_config = LoraConfig(
    lora_alpha=8,
    lora_dropout=0,
    r=32,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=modules,
)

training_arguments = TrainingArguments(
    output_dir=output_dir,                    # directory to save and repository id
    num_train_epochs=10,                       # number of training epochs
    per_device_train_batch_size=1,            # batch size per device during training
    gradient_accumulation_steps=8,            # number of steps before performing a backward/update pass
    gradient_checkpointing=True,              # use gradient checkpointing to save memory
    optim="paged_adamw_32bit",
    logging_steps=1,                         
    learning_rate=2e-4,                       # learning rate, based on QLoRA paper
    weight_decay=0.001,
    fp16=True,
    bf16=False,
    max_grad_norm=0.3,                        # max gradient norm based on QLoRA paper
    max_steps=-1,
    save_total_limit=2,
    warmup_ratio=0.03,                        # warmup ratio based on QLoRA paper
    group_by_length=False,
    lr_scheduler_type="cosine",               # use cosine learning rate scheduler
    eval_strategy="steps",                    # save checkpoint every epoch
    eval_steps = 0.1
)

trainer = SFTTrainer(
    model=model,
    args=training_arguments,
    train_dataset=train_data,
    eval_dataset=eval_data,
    peft_config=peft_config,
    dataset_text_field="text",
    tokenizer=tokenizer,
    max_seq_length=512,
    packing=False,
    dataset_kwargs={
    "add_special_tokens": False,
    "append_concat_token": False,
    }
)

/opt/conda/lib/python3.10/site-packages/huggingface_hub/utils/_deprecation.py:100: FutureWarning: Deprecated argument(s) used in '__init__': dataset_text_field, max_seq_length, dataset_kwargs. Will not be supported from version '1.0.0'.

Deprecated positional argument(s) used in SFTTrainer, please use the SFTConfig to set these arguments instead.
  warnings.warn(message, FutureWarning)
/opt/conda/lib/python3.10/site-packages/trl/trainer/sft_trainer.py:283: UserWarning: You passed a `max_seq_length` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/trl/trainer/sft_trainer.py:321: UserWarning: You passed a `dataset_text_field` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/trl/trainer/sft_trainer.py:327: UserWarning: You passed a `dataset_kwargs` argument to the SFTTrainer, the value you pass

Map:   0%|          | 0/167 [00:00<?, ? examples/s]

Map:   0%|          | 0/20 [00:00<?, ? examples/s]

/opt/conda/lib/python3.10/site-packages/trl/trainer/sft_trainer.py:396: UserWarning: You passed a tokenizer with `padding_side` not equal to `right` to the SFTTrainer. This might lead to some unexpected behaviour due to overflow issues when training a model in half-precision. You might consider adding `tokenizer.padding_side = 'right'` to your code.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/trl/trainer/sft_trainer.py:401: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `SFTTrainer.__init__`. Use `processing_class` instead.
  super().__init__(


## Model Training

In [19]:
print("Training started")

# Train model
# trainer.train()

print("Training ended")

Training started
Training ended


In [20]:
wandb.finish()
model.config.use_cache = True

## Saving the model and tokenizer

In [21]:
# Save trained model and tokenizer
trainer.save_model(output_dir)
tokenizer.save_pretrained(output_dir)

('NuminaMath-7B-TIR/tokenizer_config.json',
 'NuminaMath-7B-TIR/special_tokens_map.json',
 'NuminaMath-7B-TIR/tokenizer.json')

In [22]:
import shutil
import os

# Define your variables
dir_to_zip = output_dir

# Define the name of the output zip file
last_word = base_model_name.split('/')[-1]
output_zip = f"mullin_{output_dir}.zip"

shutil.make_archive(output_zip.replace('.zip', ''), 'zip', dir_to_zip)
print(f"Zipped contents of {dir_to_zip} into {output_zip}")

Zipped contents of NuminaMath-7B-TIR into mullin_NuminaMath-7B-TIR.zip


## Testing model after fine-tuning 

In [23]:
# s

# # Apply LoRA to the model
# model = get_peft_model(model, peft_config)

# model_path = "/kaggle/input/mullang-llama-3-1-instruct/llama-3.1-fine-tuned-model/checkpoint-540"
# # Load the fine-tuned model weights from the checkpoint
# model = PeftModel.from_pretrained(model, model_path)  # Use the checkpoint for fine-tuned weights

# # load adapter with the base model.
# #     model = model.merge_and_unload()

# model.eval()


In [24]:
sub_df = pd.read_csv('/kaggle/input/dlsprint3/test.csv')
sub_df

def generate_test_prompt_sub(data_point):
    return f"""
    {System_message}
    Here is a Bengali problem:

    Problem: {data_point["Problem"]} \nPlease solve the problem step by step and enclose the final result in \\boxed{{}}.
    label: """.strip()

sub_df['text'] = sub_df.apply(generate_test_prompt_sub, axis=1)
# sub_df = sub_df.sample(frac=0.01, random_state=42)  # random_state for reproducibility

sub_df['text_length'] = sub_df['text'].apply(lambda x: len(x.split()))  # Compute text lengths
sub_df = sub_df.sort_values(by='text_length', ascending=True).drop(columns=['text_length'])  # Sort by text length

print(len(sub_df))
sub_df

100


,ID,Problem,text
37,37,-1+2-3+4-5+6-7+8... ধারাটির প্রথম 100 টি পদের ...,You are a model smarter than Einstein when it ...
61,61,দুইটি মৌলিক সংখ্যার যোগফল 7। সংখ্যা দুইটির গুণ...,You are a model smarter than Einstein when it ...
65,65,"সর্বনিম্ন কোন বর্গসংখ্যাটি 35, 48 এবং 63 দ্বার...",You are a model smarter than Einstein when it ...
85,85,20232024+20242025 সংখ্যাটিকে 6 দিয়ে ভাগ করলে ভ...,You are a model smarter than Einstein when it ...
50,50,"$34, 35, 37, 40 \dots$ অনুক্রমটির পরবর্তী পদ ন...",You are a model smarter than Einstein when it ...
...,...,...,...
66,66,মাজেদ গণিত নিয়ে নতুন নতুন আবিষ্কার করতে খুব প...,You are a model smarter than Einstein when it ...
49,49,সাইফ গণিত নিয়ে নতুন নতুন আবিষ্কার করতে খুব পছ...,You are a model smarter than Einstein when it ...
36,36,প্রত্যয়ের কাছে একটি ম্যাজিক বক্স আছে যেখানে ক...,You are a model smarter than Einstein when it ...
30,30,$\lceil {\frac{n+(n+1)}{n+2}} \rceil + \lceil ...,You are a model smarter than Einstein when it ...


In [26]:
pipe = pipeline(task="text-generation", 
                        model=model, 
                        tokenizer=tokenizer, 
                        max_new_tokens=20, 
                        temperature=0.4)

In [29]:
prompt = sub_df.iloc[0].text
prompt

'You are a model smarter than Einstein when it comes to solving mathematical problems. You are also skilled at solving Bengali problems.\n    Here is a Bengali problem:\n\n    Problem: -1+2-3+4-5+6-7+8... ধারাটির প্রথম 100 টি পদের যোগফল কত? \nPlease solve the problem step by step and enclose the final result in \\boxed{}.\n    label:'

In [30]:
result = pipe(prompt)

/opt/conda/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:590: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.4` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(


In [31]:
result

[{'generated_text': 'You are a model smarter than Einstein when it comes to solving mathematical problems. You are also skilled at solving Bengali problems.\n    Here is a Bengali problem:\n\n    Problem: -1+2-3+4-5+6-7+8... ধারাটির প্রথম 100 টি পদের যোগফল কত? \nPlease solve the problem step by step and enclose the final result in \\boxed{}.\n    label: eq:bengali-problem\n    Solution: To solve the problem \\(-1 + '}]

In [25]:
predictions = predict(sub_df, model, tokenizer)

print("prediction ended")

prediction_file_taska = 'task_b_llm_after_training.jsonl' 
predictions_df = pd.DataFrame({'id': sub_df.id, 'label': predictions})
predictions_df.to_json(prediction_file_taska, lines=True, orient='records')
predictions_df

  0%|          | 0/100 [00:00<?, ?it/s]/opt/conda/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:590: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.4` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
  4%|▍         | 4/100 [00:12<05:10,  3.23s/it]


KeyboardInterrupt: 

In [ ]:
y_pred = predict(X_test, model, tokenizer)
evaluate(y_true, y_pred)